In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

In [2]:
from scdesigner.experimental.estimators import negative_binomial_regression
from scdesigner.experimental.transform import nullify, data_frame_mask

params = negative_binomial_regression(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
outcomes = example_sce.var_names[:4]
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = nullify(params, "coefficient", mask)
null_params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,1.733727,1.689458,1.651231,1.555795,2.025110,1.444613,1.584103,1.572423,1.389907,1.395772,...,0.622493,-0.016563,1.325979,0.732366,1.021546,-0.071549,0.967869,1.721433,-1.769911,-0.324305
"bs(pseudotime, degree=2)[1]",0.000000,0.000000,0.000000,0.000000,1.550395,1.539664,1.716174,0.460826,0.275353,-1.156779,...,0.849134,1.638570,1.285707,2.082242,-1.730549,2.030675,1.460706,-0.683348,-2.834935,2.759499
"bs(pseudotime, degree=2)[2]",0.000000,0.000000,0.000000,0.000000,-3.783759,1.599532,1.261496,1.685368,1.671482,1.665799,...,0.968832,0.368252,-1.606683,-3.385910,1.537826,0.048395,-1.933420,-3.407624,2.610776,-1.205206


In [3]:
from scdesigner.experimental.estimators import negative_binomial_copula

params = negative_binomial_copula(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
mask = data_frame_mask(params["covariance"], outcomes)
null_params = nullify(params, "covariance", mask)
null_params["covariance"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Iapp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chgb,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Rbp4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Spp1,0.395586,0.809209,-0.280929,0.299501,2.412556,-0.237144,-0.902494,0.571567,0.410007,0.327039,...,-0.101038,-0.052384,-0.008877,-0.194268,0.404364,-0.032594,-0.111565,0.026685,-0.119916,-0.012824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,0.012163,0.040434,0.239444,0.002057,-0.032594,0.132842,0.000425,0.033090,0.034472,0.051986,...,0.023667,0.086407,-0.038285,0.028245,0.002741,0.827939,0.000261,-0.018929,-0.021761,0.055698
Hes6,0.307863,0.629932,-0.105131,0.164333,-0.111565,-0.081127,-0.038200,0.358072,0.383300,0.340950,...,0.151903,-0.029840,0.403887,0.416401,0.179093,0.000261,1.160263,0.333466,-0.037000,0.079629
Serpinh1,0.455873,0.633747,-0.044600,0.288538,0.026685,0.011415,-0.159518,0.378751,0.376086,0.379002,...,0.088411,-0.004349,0.288152,0.344180,0.145530,-0.018929,0.333466,1.065656,-0.035512,0.085095
Npy,-0.411779,-0.257018,-0.156715,-0.302210,-0.119916,-0.106084,0.232162,0.067681,-0.033829,0.042039,...,0.085399,-0.015305,0.012285,-0.056983,0.055626,-0.021761,-0.037000,-0.035512,0.440396,-0.023333
